# Esercizio 5 - Topic Modeling

In questo esercizio andremo ad utilizzare la libreria gensim per fare il topic modeling, sfruttando l'algoritmo **Latent Dirichlet Allocation**
(LDA). E' un modello *non supervisionato* in quanto non necessita di un dataset annotato per fare il training, il che lo rende un algoritmo 
estremamente versatile e facile da utilizzare.

# Preparazione del dataset

In [18]:
import nltk
import pandas as pd

Tokenizzazione

In [5]:
nltk.download('punkt')

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    return tokens

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/paolobonicco/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Eliminazione delle stopwords

In [6]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

def is_stopwords(token):
    if token in en_stop:
        return True
    return False

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/paolobonicco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Lemmatizzazione

In [7]:
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

def get_lemma(token):
    return WordNetLemmatizer().lemmatize(token)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/paolobonicco/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Eliminiamo anche le parole di lunghezza <= 3

In [9]:
def transform(text):
    tokens = tokenize(summary)
    tokens = [token.lower() for token in tokens]
    tokens = [token for token in tokens if len(token) > 3]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [19]:
data = pd.read_csv("../res/data_vectorized.csv")

text_list = data["headline"].tolist()

cleaned_summary_list = [transorm(element) for element in text_list]

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte